In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def autolabel(rects):
	for rect in rects:
		height = rect.get_height()
		ax.annotate('{}'.format(height),
					xy=(rect.get_x() + rect.get_width() / 2, height),
					xytext=(0, 3),  # 3 points vertical offset
					textcoords="offset points",
					ha='center', va='bottom')

In [28]:
''' Merge csv files '''

test_dir = 'tests/test2'

list = []
dirs = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir,d))]
for d in dirs:
	d = os.path.abspath(test_dir+'/'+d)
	list += [os.path.join(d,file) for file in os.listdir(d) if file.endswith(".csv")]
print(list)

['/home/staffa/phd/tonyts/Desktop/Scripts/ConjDesc2BFGS/tests/test2/bfgs/results.csv', '/home/staffa/phd/tonyts/Desktop/Scripts/ConjDesc2BFGS/tests/test2/conj/results.csv', '/home/staffa/phd/tonyts/Desktop/Scripts/ConjDesc2BFGS/tests/test2/switch/results.csv']


In [29]:
fout=open("results2.csv","a")

# first file:
for line in open(list[0]):
	fout.write(line)
# now the rest:
for num in range(1,len(list)):
	f = open(list[num])
	f.__next__() # skip the header
	for line in f:
		fout.write(line)
	f.close() # not really needed

In [ ]:
''' Process results '''

''' No. of optimised structures per method (same no. of total structures) '''

ran_succ_list = []
rat_succ_list = []

df = pd.read_csv('results_test.csv', skipinitialspace=True)
labels = list(df['method'].unique())
dicts = {}

for method in labels:
    random = df[df['structure'].str.contains(
        "rat") == False].loc[df['method'] == method]
    rattled = df[df['structure'].str.contains(
        "rat")].loc[df['method'] == method]

    ran_succ = len([b for b in random['opt_succ'] if b])
    rat_succ = len([b for b in rattled['opt_succ'] if b])

    ran_succ_list.append(ran_succ)
    rat_succ_list.append(rat_succ)

total = len(df.loc[df['method'] == method]) / 2

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
ax.set_ylim(0,total+10) # set total samples as y limit (add space for view)
rects1 = ax.bar(x - width/2, ran_succ_list, width, label='Random')
rects2 = ax.bar(x + width/2, rat_succ_list, width, label='Rattled')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Structures')
ax.set_title('Successfully optimised structures')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

In [ ]:
''' No. of optimised structures per method (diff no. of total structures) '''
''' Normalised to per cent '''

ran_succ_list = []
rat_succ_list = []
totals = {}


df = pd.read_csv('results.csv', skipinitialspace=True)
labels = list(df['method'].unique())
dicts = {}

for method in labels:
	random = df[df['structure'].str.contains(
		"rat") == False].loc[df['method'] == method]
	rattled = df[df['structure'].str.contains(
		"rat")].loc[df['method'] == method]

	ran_succ = len([b for b in random['opt_succ'] if b])
	rat_succ = len([b for b in rattled['opt_succ'] if b])

	ran_perc = 0
	if len(random):
		ran_perc = ran_succ*100/len(random)
	rat_perc = 0
	if len(rattled):
		rat_perc = rat_succ*100/len(rattled)

	ran_succ_list.append(ran_perc)
	rat_succ_list.append(rat_perc)

y = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
fig.suptitle('Successfully optimised structures')

rects1 = ax.barh(y - width/2, ran_succ_list, width, label='Random') # x% per init category
rects2 = ax.barh(y + width/2, rat_succ_list, width, label='Rattled')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlim(0,110)
ax.set_yticks(y)
ax.set_yticklabels(labels)
ax.invert_yaxis()  # labels read top-to-bottom
ax.legend()

plt.show()

In [ ]:
''' Optimisation time per method (diff no. of total structures) '''

FONTSIZE = 10

df = pd.read_csv('results.csv', skipinitialspace=True)
methods = list(df['method'].unique())
maps = list(df['method'])
ran_succ = df[df['structure'].str.contains(
		"rat") == False].loc[df['opt_succ'] == True] # find successful optimisations
rat_succ = df[df['structure'].str.contains(
		"rat") == True].loc[df['opt_succ'] == True] 

fig, axs = plt.subplots(2,figsize=(15,15))
colors = {'blue' : methods[0], 'orange' : methods[1], 'green' : methods[2]}

for color in ['blue', 'orange', 'green']:
    x = ran_succ.loc[df['method'] == colors[color]]['structure']
    y = ran_succ.loc[df['method'] == colors[color]]['opt_time']
    axs[0].scatter(x, y, c='tab:'+color, label=colors[color],
               alpha=0.5, edgecolors='none')
axs[0].legend()
axs[0].set_xticks([])
axs[0].set_title('Random Initialisation', fontsize=FONTSIZE)


for color in ['blue', 'orange', 'green']:
    x = rat_succ.loc[df['method'] == colors[color]]['structure']
    y = rat_succ.loc[df['method'] == colors[color]]['opt_time']
    axs[1].scatter(x, y, c='tab:'+color, label=colors[color],
               alpha=0.5, edgecolors='none')
axs[1].legend()
axs[1].set_xticks([])
axs[1].set_title('Rattled Initialisation', fontsize=FONTSIZE)


fig.tight_layout()
plt.show()

In [12]:
struct_map="/users/phd/tonyts/Desktop/Data/RandomStart_Sr3Ti3O9/1.cif : structure1"
folder = struct_map.split(':')[-2].split('/')[-2]
struct = struct_map.split(':')[-1].lstrip(' ')

'structure1'

In [19]:
df = pd.read_csv('results.csv', skipinitialspace=True)
df['stdev'] = pd.Series(np.random.randn(len(df['structure'])), index=df.index)
df

,structure,method,energy,gnorm,opt_time,peak_mem,cpu_time,opt_succ,cycles,hessian,failure,options,switch,stdev
0,structure1,conj,-471.604938,0.000646,3.9144,0.55,3.9145,True,450,0,NaN,NaN,-1,-0.051909
1,structure2,conj,-468.899331,0.019964,8.9609,0.55,8.9609,False,1000,0,**** Maximum number of function calls has be...,NaN,-1,0.803651
2,structure3,conj,-468.575929,0.000578,9.1955,0.55,9.1955,True,991,0,NaN,NaN,-1,-0.231102
3,structure4,conj,-469.749708,0.001650,9.2969,0.55,9.2969,False,1000,0,**** Maximum number of function calls has be...,NaN,-1,0.799647
4,structure5,conj,-465.223134,0.126931,9.3906,0.55,9.3906,False,1000,0,**** Maximum number of function calls has be...,NaN,-1,0.474446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,rat_structure398,switch,-475.750639,0.000063,0.1187,0.60,0.1187,True,20,1,NaN,switch_minimiser bfgs gnorm 0.1\n,18,-1.932014
888,rat_structure399,switch,-475.750639,0.000073,0.1609,0.60,0.1609,True,25,1,NaN,switch_minimiser bfgs gnorm 0.1\n,20,-0.244443
889,rat_structure400,switch,-475.750639,0.000069,0.1663,0.60,0.1663,True,25,1,NaN,switch_minimiser bfgs gnorm 0.1\n,20,1.404778
890,structure13,switch,inf,0.000000,0.6794,0.56,0.6794,False,166,0,**** Too many failed attempts to optimise **...,switch_minimiser bfgs gnorm 0.1,-1,0.048765
